In [ ]:
import os
import json
import time
import hashlib
import logging
import sqlite3
from datetime import datetime, timezone
from typing import Any, Dict, Union, Optional, List
import re
from dataclasses import dataclass, asdict
from enum import Enum
import traceback

from groq import Groq

# ----------- CONFIGURATION ------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.FileHandler('multiagent_system.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Environment validation
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    logger.error("GROQ_API_KEY not set in environment variables.")
    raise ValueError("GROQ_API_KEY is required")

client = Groq(api_key=GROQ_API_KEY)

# ----------- ENUMS AND DATA STRUCTURES ------------
class InputFormat(Enum):
    PDF = "PDF"
    JSON = "JSON"
    EMAIL = "Email"
    UNKNOWN = "Unknown"

class IntentType(Enum):
    INVOICE = "Invoice"
    RFQ = "RFQ"
    COMPLAINT = "Complaint"
    REGULATION = "Regulation"
    OTHER = "Other"

@dataclass
class ClassificationResult:
    format: InputFormat
    intent: IntentType
    confidence: float = 0.0
    reasoning: str = ""

@dataclass
class ProcessingResult:
    success: bool
    data: Dict[str, Any]
    anomalies: List[str]
    processing_time: float
    agent_used: str

@dataclass
class MemoryEntry:
    conversation_id: str
    timestamp: str
    source: str
    data_type: str
    intent: str
    extracted_values: Dict[str, Any]
    agent_used: str
    processing_result: Dict[str, Any]

# ----------- ENHANCED SHARED MEMORY ------------
class SharedMemory:
    def __init__(self, db_path: str = "multiagent_memory.db"):
        self.db_path = db_path
        self._init_database()
        logger.info(f"SharedMemory initialized with database: {db_path}")

    def _init_database(self):
        """Initialize SQLite database for persistent memory"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute("""
                    CREATE TABLE IF NOT EXISTS memory_entries (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        conversation_id TEXT NOT NULL,
                        timestamp TEXT NOT NULL,
                        source TEXT NOT NULL,
                        data_type TEXT NOT NULL,
                        intent TEXT NOT NULL,
                        extracted_values TEXT NOT NULL,
                        agent_used TEXT NOT NULL,
                        processing_result TEXT NOT NULL,
                        created_at DATETIME DEFAULT CURRENT_TIMESTAMP
                    )
                """)
                conn.execute("""
                    CREATE INDEX IF NOT EXISTS idx_conversation_id 
                    ON memory_entries(conversation_id)
                """)
                conn.execute("""
                    CREATE INDEX IF NOT EXISTS idx_timestamp 
                    ON memory_entries(timestamp)
                """)
                conn.commit()
        except Exception as e:
            logger.error(f"Database initialization failed: {e}")
            raise

    def log(self, entry: MemoryEntry) -> bool:
        """Log entry to persistent storage"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute("""
                    INSERT INTO memory_entries 
                    (conversation_id, timestamp, source, data_type, intent, 
                     extracted_values, agent_used, processing_result)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    entry.conversation_id,
                    entry.timestamp,
                    entry.source,
                    entry.data_type,
                    entry.intent,
                    json.dumps(entry.extracted_values),
                    entry.agent_used,
                    json.dumps(entry.processing_result)
                ))
                conn.commit()
            logger.debug(f"Logged entry for conversation {entry.conversation_id}")
            return True
        except Exception as e:
            logger.error(f"Failed to log memory entry: {e}")
            return False

    def get_context(self, conversation_id: str, limit: int = 10) -> List[MemoryEntry]:
        """Retrieve conversation context"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.execute("""
                    SELECT conversation_id, timestamp, source, data_type, intent,
                           extracted_values, agent_used, processing_result
                    FROM memory_entries
                    WHERE conversation_id = ?
                    ORDER BY timestamp DESC
                    LIMIT ?
                """, (conversation_id, limit))
                
                entries = []
                for row in cursor.fetchall():
                    entries.append(MemoryEntry(
                        conversation_id=row[0],
                        timestamp=row[1],
                        source=row[2],
                        data_type=row[3],
                        intent=row[4],
                        extracted_values=json.loads(row[5]),
                        agent_used=row[6],
                        processing_result=json.loads(row[7])
                    ))
                return entries
        except Exception as e:
            logger.error(f"Failed to retrieve context: {e}")
            return []

    def get_last_extraction(self, conversation_id: str) -> Dict[str, Any]:
        """Get the most recent extraction for context"""
        entries = self.get_context(conversation_id, limit=1)
        return entries[0].extracted_values if entries else {}

    def get_conversation_summary(self, conversation_id: str) -> Dict[str, Any]:
        """Get summarized conversation context"""
        entries = self.get_context(conversation_id)
        if not entries:
            return {}
        
        summary = {
            "total_interactions": len(entries),
            "formats_processed": list(set(entry.data_type for entry in entries)),
            "intents_detected": list(set(entry.intent for entry in entries)),
            "agents_used": list(set(entry.agent_used for entry in entries)),
            "latest_timestamp": entries[0].timestamp if entries else None,
            "latest_extraction": entries[0].extracted_values if entries else {}
        }
        return summary

shared_memory = SharedMemory()

# ----------- ENHANCED GROQ COMPLETION UTILS ------------
def groq_completion(
    prompt: str,
    system: str = "",
    model: str = "llama3-70b-8192",
    response_format: str = None,
    max_retries: int = 3,
    timeout: int = 30,
    temperature: float = 0.3
) -> Any:
    """Enhanced Groq completion with better error handling and validation"""
    
    if not prompt.strip():
        raise ValueError("Prompt cannot be empty")
    
    if response_format == "json" and "json" not in prompt.lower():
        prompt += "\n\nPlease respond in valid JSON format only."

    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})

    last_error = None
    for attempt in range(1, max_retries + 1):
        try:
            logger.debug(f"Groq call attempt {attempt}/{max_retries}")
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                response_format={"type": "json_object"} if response_format == "json" else None,
                timeout=timeout,
            )
            
            content = response.choices[0].message.content
            if not content:
                raise ValueError("Empty response from Groq")
                
            # Validate JSON if requested
            if response_format == "json":
                try:
                    json.loads(content)
                except json.JSONDecodeError as e:
                    raise ValueError(f"Invalid JSON response: {e}")
                    
            return content
            
        except Exception as e:
            last_error = e
            logger.warning(f"Groq completion failed (attempt {attempt}/{max_retries}): {e}")
            if attempt < max_retries:
                time.sleep(2 ** attempt)
    
    logger.error(f"Groq completion failed after {max_retries} retries: {last_error}")
    raise RuntimeError(f"Groq completion failed: {last_error}")

# ----------- ENHANCED CLASSIFIER AGENT ------------
def classify_agent(input_data: Union[str, bytes]) -> ClassificationResult:
    """Enhanced classification with confidence scoring and reasoning"""
    
    try:
        # PDF detection
        if isinstance(input_data, bytes) and input_data.startswith(b'%PDF'):
            return ClassificationResult(
                format=InputFormat.PDF,
                intent=IntentType.OTHER,  # Will be determined after text extraction
                confidence=1.0,
                reasoning="PDF magic number detected"
            )
        
        # Convert bytes to string if needed
        if isinstance(input_data, bytes):
            try:
                content = input_data.decode('utf-8', errors='ignore')[:3000]
            except Exception:
                content = str(input_data)[:3000]
        else:
            content = str(input_data)[:3000]

        # Enhanced JSON detection
        is_json = False
        try:
            json.loads(content)
            is_json = True
        except (json.JSONDecodeError, TypeError):
            pass

        system_prompt = """
        You are an expert document classifier. Analyze the input and classify it with high accuracy.
        
        Format Classification:
        - PDF: Documents with PDF structure
        - JSON: Valid JSON data structures
        - Email: Email content, messages, or communication text
        
        Intent Classification:
        - Invoice: Billing documents, payment requests, financial statements
        - RFQ: Request for Quote, procurement requests, vendor inquiries
        - Complaint: Customer complaints, issues, grievances
        - Regulation: Legal documents, compliance materials, policy documents
        - Other: General content that doesn't fit above categories
        
        Provide confidence score (0.0-1.0) and reasoning for your classification.
        """
        
        prompt = f"""
        Content to classify:
        {content}
        
        Analysis requirements:
        1. Determine format (PDF/JSON/Email)
        2. Determine intent (Invoice/RFQ/Complaint/Regulation/Other)
        3. Provide confidence score (0.0-1.0)
        4. Provide reasoning for classification
        
        Output JSON with keys: 'format', 'intent', 'confidence', 'reasoning'
        """

        raw = groq_completion(prompt, system_prompt, response_format="json")
        result = json.loads(raw)
        
        # Override format if JSON was detected programmatically
        if is_json:
            result['format'] = 'JSON'
            result['confidence'] = max(result.get('confidence', 0.5), 0.9)
        
        classification = ClassificationResult(
            format=InputFormat(result.get('format', 'Email')),
            intent=IntentType(result.get('intent', 'Other')),
            confidence=float(result.get('confidence', 0.5)),
            reasoning=result.get('reasoning', 'No reasoning provided')
        )
        
        logger.info(f"Classification: {classification}")
        return classification
        
    except Exception as e:
        logger.error(f"Classification error: {e}")
        return ClassificationResult(
            format=InputFormat.EMAIL,
            intent=IntentType.OTHER,
            confidence=0.1,
            reasoning=f"Classification failed: {str(e)}"
        )

# ----------- ENHANCED JSON AGENT ------------
def json_agent(payload: dict, conversation_id: str, intent: IntentType) -> ProcessingResult:
    """Enhanced JSON processing with schema validation and anomaly detection"""
    
    start_time = time.time()
    
    try:
        # Get conversation context
        context_summary = shared_memory.get_conversation_summary(conversation_id)
        
        # Define target schemas based on intent
        schema_definitions = {
            IntentType.INVOICE: {
                "required_fields": ["amount", "date", "vendor", "invoice_number"],
                "optional_fields": ["tax_amount", "due_date", "description", "line_items"]
            },
            IntentType.RFQ: {
                "required_fields": ["items", "quantity", "deadline", "requester"],
                "optional_fields": ["specifications", "budget", "delivery_location"]
            },
            IntentType.COMPLAINT: {
                "required_fields": ["issue", "customer", "date"],
                "optional_fields": ["severity", "resolution_requested", "previous_interactions"]
            }
        }
        
        target_schema = schema_definitions.get(intent, {
            "required_fields": ["type", "content"],
            "optional_fields": ["metadata", "source"]
        })
        
        system_prompt = f"""
        You are a JSON processing expert specializing in {intent.value} documents.
        Extract and validate data according to the target schema.
        
        Target Schema:
        - Required fields: {target_schema['required_fields']}
        - Optional fields: {target_schema['optional_fields']}
        
        Flag any anomalies such as:
        - Missing required fields
        - Invalid data types
        - Suspicious values
        - Inconsistent data
        """
        
        prompt = f"""
        Conversation Context Summary:
        {json.dumps(context_summary, indent=2)}
        
        Input JSON Payload:
        {json.dumps(payload, indent=2)}
        
        Tasks:
        1. Extract data matching the target schema
        2. Validate all fields
        3. Flag any anomalies or errors
        4. Provide data quality score (0.0-1.0)
        
        Output JSON with keys: 'extracted_data', 'anomalies', 'data_quality_score'
        """

        raw = groq_completion(prompt, system_prompt, response_format="json")
        result = json.loads(raw)
        
        processing_time = time.time() - start_time
        
        return ProcessingResult(
            success=True,
            data=result.get('extracted_data', {}),
            anomalies=result.get('anomalies', []),
            processing_time=processing_time,
            agent_used="JSON_Agent"
        )
        
    except Exception as e:
        processing_time = time.time() - start_time
        logger.error(f"JSON agent error: {e}")
        return ProcessingResult(
            success=False,
            data={},
            anomalies=[f"Processing failed: {str(e)}"],
            processing_time=processing_time,
            agent_used="JSON_Agent"
        )

# ----------- ENHANCED EMAIL AGENT ------------
def email_agent(content: str, conversation_id: str, intent: IntentType) -> ProcessingResult:
    """Enhanced email processing with intent-specific extraction"""
    
    start_time = time.time()
    
    try:
        # Clean and preprocess content
        cleaned_content = re.sub(r'\s+', ' ', content.strip())
        if len(cleaned_content) > 8000:
            cleaned_content = cleaned_content[:8000] + "... [truncated]"
        
        # Get conversation context
        context_summary = shared_memory.get_conversation_summary(conversation_id)
        
        # Intent-specific extraction prompts
        intent_prompts = {
            IntentType.INVOICE: "Focus on extracting billing information, amounts, dates, and vendor details.",
            IntentType.RFQ: "Focus on extracting requested items, quantities, deadlines, and specifications.",
            IntentType.COMPLAINT: "Focus on extracting the issue, customer information, and resolution requests.",
            IntentType.REGULATION: "Focus on extracting regulatory requirements, compliance details, and deadlines."
        }
        
        specific_prompt = intent_prompts.get(intent, "Extract key information relevant to this communication.")
        
        system_prompt = f"""
        You are an email processing expert specializing in {intent.value} communications.
        Extract comprehensive information with high accuracy.
        
        {specific_prompt}
        
        Always extract:
        - Sender information
        - Urgency level (Low/Medium/High/Critical)
        - Key topics and entities
        - Action items or requests
        - Summary of content
        """
        
        prompt = f"""
        Conversation Context Summary:
        {json.dumps(context_summary, indent=2) if context_summary else "No previous context"}
        
        Email Content:
        {cleaned_content}
        
        Extract and structure the following information:
        1. sender: Email sender details
        2. urgency: Urgency level (Low/Medium/High/Critical)
        3. topics: List of main topics discussed
        4. action_items: List of requested actions
        5. summary: Comprehensive summary
        6. entities: Named entities (people, organizations, locations)
        7. intent_specific_data: Data specific to {intent.value}
        8. confidence_score: Extraction confidence (0.0-1.0)
        
        Output JSON with these exact keys.
        """

        raw = groq_completion(
            prompt, 
            system_prompt, 
            model="llama3-70b-8192",
            response_format="json"
        )
        
        result = json.loads(raw)
        processing_time = time.time() - start_time
        
        # Validate extracted data
        anomalies = []
        if not result.get('sender'):
            anomalies.append("No sender information found")
        if not result.get('summary'):
            anomalies.append("No summary generated")
        if result.get('confidence_score', 0) < 0.5:
            anomalies.append("Low extraction confidence")
        
        return ProcessingResult(
            success=True,
            data=result,
            anomalies=anomalies,
            processing_time=processing_time,
            agent_used="Email_Agent"
        )
        
    except Exception as e:
        processing_time = time.time() - start_time
        logger.error(f"Email agent error: {e}")
        return ProcessingResult(
            success=False,
            data={"error": str(e)},
            anomalies=[f"Processing failed: {str(e)}"],
            processing_time=processing_time,
            agent_used="Email_Agent"
        )

# ----------- ENHANCED PDF AGENT ------------
def pdf_agent(pdf_bytes: bytes, conversation_id: str) -> ProcessingResult:
    """Enhanced PDF processing with better text extraction and error handling"""
    
    start_time = time.time()
    
    try:
        # Try multiple PDF libraries for robustness
        text = ""
        extraction_method = "unknown"
        
        # Method 1: PyPDF2
        try:
            from PyPDF2 import PdfReader
            from io import BytesIO
            
            reader = PdfReader(BytesIO(pdf_bytes))
            for page_num, page in enumerate(reader.pages):
                page_text = page.extract_text() or ""
                text += f"[Page {page_num + 1}]\n{page_text}\n"
                if len(text) > 15000:  # Reasonable limit
                    break
            extraction_method = "PyPDF2"
            
        except ImportError:
            logger.warning("PyPDF2 not available, trying alternative methods")
        except Exception as e:
            logger.warning(f"PyPDF2 extraction failed: {e}")
        
        # Method 2: pdfplumber (if available)
        if not text:
            try:
                import pdfplumber
                from io import BytesIO
                
                with pdfplumber.open(BytesIO(pdf_bytes)) as pdf:
                    for page_num, page in enumerate(pdf.pages):
                        page_text = page.extract_text() or ""
                        text += f"[Page {page_num + 1}]\n{page_text}\n"
                        if len(text) > 15000:
                            break
                extraction_method = "pdfplumber"
                
            except ImportError:
                logger.warning("pdfplumber not available")
            except Exception as e:
                logger.warning(f"pdfplumber extraction failed: {e}")
        
        if not text:
            raise ValueError("No text could be extracted from PDF")
        
        logger.info(f"Extracted {len(text)} characters from PDF using {extraction_method}")
        
        # Re-classify the extracted text to determine intent
        classification = classify_agent(text[:2000])
        intent = classification.intent
        
        # Process extracted text as email content
        email_result = email_agent(text, conversation_id, intent)
        
        # Enhance with PDF-specific metadata
        pdf_metadata = {
            "extraction_method": extraction_method,
            "text_length": len(text),
            "estimated_pages": text.count("[Page"),
            "classification_confidence": classification.confidence
        }
        
        if email_result.success:
            email_result.data["pdf_metadata"] = pdf_metadata
        
        email_result.agent_used = "PDF_Agent"
        email_result.processing_time = time.time() - start_time
        
        return email_result
        
    except ImportError as e:
        processing_time = time.time() - start_time
        logger.error("PDF processing libraries not available")
        return ProcessingResult(
            success=False,
            data={"error": "PDF processing libraries not installed"},
            anomalies=["PyPDF2 or pdfplumber required for PDF processing"],
            processing_time=processing_time,
            agent_used="PDF_Agent"
        )
    except Exception as e:
        processing_time = time.time() - start_time
        logger.exception("PDF processing failed")
        return ProcessingResult(
            success=False,
            data={"error": str(e)},
            anomalies=[f"PDF processing failed: {str(e)}"],
            processing_time=processing_time,
            agent_used="PDF_Agent"
        )

# ----------- ENHANCED MAIN ORCHESTRATOR ------------
def process_input(
    input_data: Any, 
    conversation_id: Optional[str] = None,
    metadata: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Enhanced main orchestrator with comprehensive error handling and logging
    """
    
    if not conversation_id:
        conversation_id = hashlib.md5(f"{time.time()}{str(input_data)[:100]}".encode()).hexdigest()[:12]
    
    start_time = time.time()
    metadata = metadata or {}
    
    try:
        logger.info(f"Processing input for conversation {conversation_id}")
        
        # Step 1: Enhanced Classification
        classification = classify_agent(input_data)
        fmt = classification.format
        intent = classification.intent
        
        logger.info(f"Classified as {fmt.value} with intent {intent.value} (confidence: {classification.confidence})")
        
        # Step 2: Route to appropriate agent with enhanced error handling
        result = None
        
        if fmt == InputFormat.JSON:
            try:
                if isinstance(input_data, str):
                    payload = json.loads(input_data)
                elif isinstance(input_data, dict):
                    payload = input_data
                else:
                    payload = {"raw_data": str(input_data)[:1000]}
            except (TypeError, json.JSONDecodeError) as e:
                logger.warning(f"JSON parsing failed: {e}")
                payload = {"raw_data": str(input_data)[:1000], "parse_error": str(e)}
            
            result = json_agent(payload, conversation_id, intent)
            
        elif fmt == InputFormat.PDF:
            if not isinstance(input_data, bytes):
                try:
                    input_data = input_data.encode('utf-8')
                except Exception:
                    input_data = str(input_data).encode('utf-8', errors='ignore')
            
            result = pdf_agent(input_data, conversation_id)
            
        else:  # Email or fallback
            if isinstance(input_data, bytes):
                try:
                    content = input_data.decode('utf-8', errors='ignore')
                except Exception:
                    content = str(input_data)
            else:
                content = str(input_data)
            
            result = email_agent(content, conversation_id, intent)
        
        # Step 3: Log to shared memory
        memory_entry = MemoryEntry(
            conversation_id=conversation_id,
            timestamp=datetime.now(timezone.utc).isoformat(),
            source="user_input",
            data_type=fmt.value,
            intent=intent.value,
            extracted_values=result.data if result else {},
            agent_used=result.agent_used if result else "None",
            processing_result=asdict(result) if result else {}
        )
        
        shared_memory.log(memory_entry)
        
        # Step 4: Compile comprehensive response
        total_processing_time = time.time() - start_time
        
        response = {
            "conversation_id": conversation_id,
            "classification": {
                "format": fmt.value,
                "intent": intent.value,
                "confidence": classification.confidence,
                "reasoning": classification.reasoning
            },
            "processing_result": asdict(result) if result else {},
            "context": {
                "conversation_summary": shared_memory.get_conversation_summary(conversation_id),
                "total_interactions": len(shared_memory.get_context(conversation_id))
            },
            "performance": {
                "total_processing_time": total_processing_time,
                "agent_processing_time": result.processing_time if result else 0
            },
            "metadata": metadata,
            "status": "success" if result and result.success else "partial_failure"
        }
        
        logger.info(f"Processing completed for {conversation_id} in {total_processing_time:.2f}s")
        return response
        
    except Exception as e:
        total_processing_time = time.time() - start_time
        logger.exception(f"Processing failed for conversation {conversation_id}: {e}")
        
        error_response = {
            "conversation_id": conversation_id,
            "error": str(e),
            "traceback": traceback.format_exc(),
            "status": "error",
            "performance": {
                "total_processing_time": total_processing_time
            },
            "metadata": metadata
        }
        
        return error_response

# ----------- UTILITY FUNCTIONS ------------
def get_conversation_history(conversation_id: str) -> Dict[str, Any]:
    """Get full conversation history with analytics"""
    entries = shared_memory.get_context(conversation_id, limit=100)
    summary = shared_memory.get_conversation_summary(conversation_id)
    
    return {
        "conversation_id": conversation_id,
        "summary": summary,
        "entries": [asdict(entry) for entry in entries],
        "total_entries": len(entries)
    }

def health_check() -> Dict[str, Any]:
    """System health check"""
    try:
        # Test Groq connection
        test_response = groq_completion("Hello", response_format="json", max_retries=1)
        groq_status = "healthy"
    except Exception as e:
        groq_status = f"unhealthy: {str(e)}"
    
    # Test database connection
    try:
        test_entry = MemoryEntry(
            conversation_id="health_check",
            timestamp=datetime.now(timezone.utc).isoformat(),
            source="system",
            data_type="test",
            intent="health_check",
            extracted_values={},
            agent_used="system",
            processing_result={}
        )
        db_status = "healthy" if shared_memory.log(test_entry) else "unhealthy"
    except Exception as e:
        db_status = f"unhealthy: {str(e)}"
    
    return {
        "status": "healthy" if groq_status == "healthy" and db_status == "healthy" else "degraded",
        "components": {
            "groq_api": groq_status,
            "database": db_status
        },
        "timestamp": datetime.now(timezone.utc).isoformat()
    }

if __name__ == "__main__":
    # 1. PDF (bytes)
    with open("sample.pdf", "rb") as f:
        pdf_bytes = f.read()
    pdf_result = process_input(pdf_bytes, conversation_id="test_pdf")
    print("=== PDF RESULT ===")
    print(json.dumps(pdf_result, indent=2))

    # 2. JSON (dict or string)
    with open("sample.json", "r", encoding="utf-8") as f:
        json_payload = json.load(f)
    # You can either pass the dict directly...
    json_result = process_input(json_payload, conversation_id="test_json")
    # ...or read it as raw text:
    # raw_json_str = json.dumps(json_payload)
    # json_result = process_input(raw_json_str, conversation_id="test_json")
    print("\n=== JSON RESULT ===")
    print(json.dumps(json_result, indent=2))

    # 3. Plain text (bytes or str)
    # If you have sample.txt as UTF-8:
    with open("sample.txt", "r", encoding="utf-8") as f:
        text_content = f.read()
    text_result = process_input(text_content, conversation_id="test_txt")
    print("\n=== TEXT RESULT ===")
    print(json.dumps(text_result, indent=2))


2025-05-30 11:25:29 [INFO] __main__: SharedMemory initialized with database: multiagent_memory.db
2025-05-30 11:25:29 [INFO] __main__: Processing input for conversation 6dbe6ee95a8a
2025-05-30 11:25:29 [INFO] httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-30 11:25:29 [INFO] __main__: Classification: ClassificationResult(format=<InputFormat.EMAIL: 'Email'>, intent=<IntentType.COMPLAINT: 'Complaint'>, confidence=0.95, reasoning='The content contains a clear subject line indicating a product complaint, and the body of the email describes a specific issue with a purchased product, which strongly suggests a complaint. The tone is also urgent, which is consistent with a complaint intent.')
2025-05-30 11:25:29 [INFO] __main__: Classified as Email with intent Complaint (confidence: 0.95)
2025-05-30 11:25:30 [INFO] httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-30 11:25:30 [INFO] __main__: 

{
  "conversation_id": "6dbe6ee95a8a",
  "classification": {
    "format": "Email",
    "intent": "Complaint",
    "confidence": 0.95,
    "reasoning": "The content contains a clear subject line indicating a product complaint, and the body of the email describes a specific issue with a purchased product, which strongly suggests a complaint. The tone is also urgent, which is consistent with a complaint intent."
  },
  "processing_result": {
    "success": true,
    "data": {
      "sender": {
        "email": "customer@example.com",
        "name": "John Doe"
      },
      "urgency": "High",
      "topics": [
        "Product Complaint",
        "Laptop Issue"
      ],
      "action_items": [
        "Resolve the issue urgently"
      ],
      "summary": "Customer is complaining about a laptop screen flickering issue within 3 days of purchase and requests urgent resolution.",
      "entities": {
        "person": "John Doe",
        "organization": null,
        "location": null
      